---

### 📜 Licence d'utilisation

Ce document est protégé sous licence **Creative Commons BY-NC-ND 4.0 International**  
🔒 **Aucune modification ni réutilisation sans autorisation explicite de l'auteur.**

- 👤 Auteur : Christie Vassilian  
- 📥 Téléchargement autorisé uniquement à usage pédagogique personnel  
- 🚫 Réutilisation commerciale ou modification interdite  

[![Licence CC BY-NC-ND](https://licensebuttons.net/l/by-nc-nd/4.0/88x31.png)](https://creativecommons.org/licenses/by-nc-nd/4.0/)

---

# YOLOv8 – Découvrir la détection d’objets  
_Notebook 1 : images, vidéos, webcam_

## Objectifs de la séance

Dans ce notebook, tu vas :

1. Charger un **modèle YOLOv8 pré-entraîné** (détection d’objets).
2. L’utiliser sur :
   - une **image**,
   - une **vidéo**,
   - la **webcam**.
3. Lire et interpréter les résultats : **boîtes**, **noms d’objets**, **scores de confiance**.
4. Réfléchir aux **forces** et **limites** de ce type d’IA.

Des **questions** te guideront au fur et à mesure. Prends le temps d’y répondre dans ta copie ou dans un fichier texte à part.


---
## 1. Préparation de l’environnement

### 1.1. Installation des bibliothèques

> ⚠️ Cette cellule est à exécuter **une seule fois** pour installer les bibliothèques nécessaires dans l’environnement.

**Question 1 :**  
Avant d’exécuter la cellule suivante, réponds :  
> À ton avis, à quoi servent les bibliothèques suivantes ?  
> - `ultralytics`  
> - `opencv-python` (`cv2`)  
> - `matplotlib`

*(Tu peux chercher rapidement sur Internet si besoin.)*


In [ ]:
# À exécuter une seule fois (ou à commenter si déjà installé)
!pip install -q ultralytics opencv-python matplotlib


### 1.2. Import des bibliothèques

On importe maintenant les modules Python dont on aura besoin.


In [ ]:
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

print("Versions :")
print(" - OpenCV :", cv2.__version__)
print(" - YOLO (ultralytics) importé avec succès")


### 1.3. Chargement du modèle YOLOv8

Nous allons utiliser un modèle **pré-entraîné** : `yolov8n.pt` (“n” pour *nano* → modèle léger, plus rapide).

**Question 2 :**  
Explique avec tes mots ce que signifie *« modèle pré-entraîné »*.  
En quoi est-ce différent d’un programme “classique” écrit à la main ?


In [ ]:
# Chargement du modèle YOLOv8 "nano"
model = YOLO("yolov8n.pt")
print("Modèle YOLOv8n chargé.")


---
## 2. Détection d’objets sur une image

Nous allons d’abord tester YOLO sur une **image fixe**.

> Prépare dans le même dossier que ce notebook un ou plusieurs fichiers image, par exemple :  
> - `classe.jpg` (photo d’une salle de classe)  
> - `rue.jpg` (photo d’une rue)  
> - ou une autre image libre de droit.

Nous allons commencer avec un nom de fichier par défaut que tu pourras modifier.


In [ ]:
# Nom du fichier image à analyser
# Modifie cette ligne avec le nom de ton image, par ex. "classe.jpg"
image_path = Path("image_test.jpg")

if not image_path.is_file():
    print(f"⚠️ Fichier {image_path} introuvable. Place une image dans le dossier et change le nom ci-dessus.")
else:
    print(f"Image trouvée : {image_path}")


In [ ]:
def show_image_bgr(image_bgr, title="Image"):
    """Affiche une image OpenCV (BGR) avec matplotlib (RGB)."""
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(8, 6))
    plt.imshow(image_rgb)
    plt.axis("off")
    plt.title(title)
    plt.show()

if image_path.is_file():
    img_bgr = cv2.imread(str(image_path))
    if img_bgr is None:
        print("Erreur de lecture de l'image.")
    else:
        show_image_bgr(img_bgr, title=f"Image originale : {image_path.name}")


### 2.1. Lancer YOLO sur l’image

Nous allons maintenant appliquer le modèle YOLO à l’image et afficher le résultat annoté.

**Question 3 :**  
Avant de lancer la détection, devine :  
> Quels objets YOLO va-t-il probablement détecter sur cette image ?  
> Fais une petite liste de 3 à 5 objets possibles.


In [ ]:
if image_path.is_file():
    # Lancement de la détection
    results = model(source=str(image_path))  # on donne directement le chemin du fichier
    
    # On récupère le premier (et unique) résultat
    result = results[0]
    
    # Image annotée par YOLO (numpy array BGR)
    annotated_frame = result.plot()
    
    show_image_bgr(annotated_frame, title=f"Image annotée par YOLO : {image_path.name}")
else:
    print("Aucune image à analyser. Vérifie le chemin du fichier plus haut.")


### 2.2. Lire les résultats (classes, scores…)

YOLO ne donne pas seulement l’image annotée. Il renvoie aussi une **liste d’objets détectés**, chaque objet étant décrit par :

- sa **classe** (par ex. `person`, `car`, `bottle`),  
- un **score de confiance** entre 0 et 1,  
- ses coordonnées dans l’image (rectangle).

Nous allons afficher cette liste de manière lisible.


In [ ]:
if image_path.is_file():
    result = results[0]
    boxes = result.boxes  # boîtes détectées
    
    class_ids = boxes.cls.cpu().numpy().astype(int)
    scores = boxes.conf.cpu().numpy()
    xyxy = boxes.xyxy.cpu().numpy()  # coordonnées : x1, y1, x2, y2
    
    names = result.names  # dictionnaire id -> nom de classe
    
    print("Objets détectés :")
    for cls_id, score, (x1, y1, x2, y2) in zip(class_ids, scores, xyxy):
        label = names[int(cls_id)]
        print(f" - {label:10s} | score = {score:.2f} | bbox = ({int(x1)}, {int(y1)}, {int(x2)}, {int(y2)})")
else:
    print("Aucun résultat à afficher.")


**Question 4 :**  
1. Combien d’objets sont détectés au total ?  
2. Quels sont les **3 objets** avec les plus grands scores de confiance ?  
3. Repère un cas où l’IA se trompe (ou semble se tromper). Est-ce un **faux positif** (objet détecté mais inexistant) ou un **faux négatif** (objet réel non détecté) ?

---

## 3. YOLO sur une vidéo

Nous allons maintenant appliquer YOLO à une **vidéo**.

> Prépare un petit fichier vidéo (par exemple `video_test.mp4`) dans le même dossier que ce notebook.  
> Idéalement une vidéo courte (quelques secondes) avec des personnes ou des objets en mouvement.


In [ ]:
# Nom du fichier vidéo à analyser
video_path = Path("video_test.mp4")

if not video_path.is_file():
    print(f"⚠️ Fichier vidéo {video_path} introuvable. Place une vidéo dans le dossier et change le nom ci-dessus.")
else:
    print(f"Vidéo trouvée : {video_path}")


### 3.1. Traitement de la vidéo image par image

Pour une vidéo, YOLO fonctionne **image par image** :  
chaque image (frame) est traitée comme une photo.

Nous allons :  
1. ouvrir la vidéo,  
2. lire les images une par une,  
3. appliquer YOLO sur chaque image,  
4. afficher **quelques images annotées** (une toutes les n images pour éviter un affichage trop lourd).

**Question 5 :**  
À ton avis, pourquoi la détection est-elle plus difficile sur une vidéo que sur une image fixe ?  
Donne au moins **2 raisons** possibles.


In [ ]:
if video_path.is_file():
    cap = cv2.VideoCapture(str(video_path))
    
    if not cap.isOpened():
        print("Erreur : impossible d'ouvrir la vidéo.")
    else:
        frame_count = 0
        max_frames_to_show = 5   # nombre maximum d'images à afficher dans le notebook
        step = 10                # on affiche 1 image sur 10
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break  # fin de la vidéo
            
            frame_count += 1
            
            # On ne traite qu'une image sur 'step' pour l'affichage
            if frame_count % step != 0:
                continue
            
            # Application de YOLO sur la frame
            results_video = model(source=frame, verbose=False)
            result_v = results_video[0]
            annotated = result_v.plot()  # image annotée
            
            show_image_bgr(annotated, title=f"Frame {frame_count} annotée")
            
            if frame_count // step >= max_frames_to_show:
                break
        
        cap.release()
        print("Traitement vidéo terminé.")
else:
    print("Aucune vidéo à analyser. Vérifie le chemin du fichier plus haut.")


**Question 6 :**  
1. Sur les images annotées de la vidéo, est-ce que YOLO détecte **toujours** les mêmes objets ?  
2. Donne un exemple où un objet est détecté sur une image mais pas sur la suivante.  
3. Explique pourquoi cela peut poser problème si l’on veut **compter** les objets ou **suivre** une personne.

> Remarque : YOLO **détecte** les objets, mais ne fait pas de **suivi** (tracking) d’une image à l’autre. Le suivi est un autre problème à part.

---

## 4. YOLO en direct avec la webcam

Dernière étape : faire tourner YOLO en **temps réel** sur la webcam.

> ⚠️ Cette partie fonctionne surtout si tu exécutes le notebook en local (par exemple avec Anaconda / VS Code), pas forcément dans un environnement en ligne.

Le principe :  
1. Ouvrir la webcam (`cv2.VideoCapture(0)`),  
2. Lire en boucle les images,  
3. Appliquer YOLO sur chaque image,  
4. Afficher le résultat dans une fenêtre,  
5. Quitter quand l’utilisateur appuie sur la touche `q`.


In [ ]:
use_webcam = False  # Passe à True si tu es sur un ordinateur local avec webcam

if use_webcam:
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Erreur : impossible d'ouvrir la webcam.")
    else:
        print("Appuie sur 'q' dans la fenêtre vidéo pour quitter.")
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # Application de YOLO sur la frame
            results_cam = model(source=frame, verbose=False)
            result_c = results_cam[0]
            annotated = result_c.plot()
            
            cv2.imshow("Webcam YOLOv8", annotated)
            
            # Quitter avec la touche 'q'
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        cap.release()
        cv2.destroyAllWindows()
else:
    print("La webcam n'est pas activée. Passe use_webcam = True pour tester en local.")


**Question 7 :**  
En testant sur la webcam (si possible) :  
1. Est-ce que YOLO te détecte correctement comme `person` ?  
2. Que se passe-t-il si tu te rapproches très près de la caméra ? si tu t’éloignes beaucoup ?  
3. Que se passe-t-il si tu changes l’éclairage (lumière forte, lumière faible) ?

Explique en quelques phrases pourquoi la **qualité des images** (lumière, angle, distance) est essentielle pour les performances de YOLO.

---

## 5. Bilan : que fait vraiment YOLO ?

Réponds aux questions suivantes en t’appuyant sur ce que tu as observé :

**Question 8 :**  
1. YOLO “comprend-il” ce qu’il voit comme un humain, ou bien fait-il autre chose ?  
2. Pourquoi peut-on dire qu’il **reconnaît des motifs statistiques** plutôt que des “objets” au sens humain ?

**Question 9 :**  
Donne un exemple de **situation réelle** où YOLO pourrait être utile, et un exemple où son utilisation peut être **dangereuse ou trompeuse** s’il se trompe (faux positifs ou faux négatifs).

**Question 10 :**  
Propose une idée d’amélioration ou d’extension :  
- Que voudrais-tu mesurer, compter ou analyser en plus ?  
- Sur quelles images ou vidéos aimerais-tu tester YOLO ?

---

Fin du Notebook 1 : tu as découvert comment utiliser YOLOv8 sur des images, des vidéos et la webcam, et tu as commencé à réfléchir à ses limites et à ses usages possibles.


---

### 📜 Licence d'utilisation

Ce document est protégé sous licence **Creative Commons BY-NC-ND 4.0 International**  
🔒 **Aucune modification ni réutilisation sans autorisation explicite de l'auteur.**

- 👤 Auteur : Christie Vassilian  
- 📥 Téléchargement autorisé uniquement à usage pédagogique personnel  
- 🚫 Réutilisation commerciale ou modification interdite  

[![Licence CC BY-NC-ND](https://licensebuttons.net/l/by-nc-nd/4.0/88x31.png)](https://creativecommons.org/licenses/by-nc-nd/4.0/)

---